In [1]:
import numpy as np
import pandas as pd

In [2]:
#Fuentes Información

df= pd.read_excel("C:/Renato/script_atenea/Split/SI.xlsx"
,sheet_name="Hoja1")

df_forecast= pd.read_excel("C:/Renato/script_atenea/Split/Output_x.xlsx"
,sheet_name="Hoja1")

df_demanda= pd.read_excel("C:/Renato/script_atenea/Split2/IBP_FAM_OFI.xlsx"
,sheet_name="Hoja1")

In [3]:
#DICCIONARIOS y LISTAS

dicc_categoría=df[["Cat2","CATEGORIA_AFO"]].drop_duplicates()

Categorias = df["CATEGORIA_AFO"].unique().tolist()

familias =df['FAMILIA_AFO'].unique().tolist()

Canales =df['CANAL'].unique().tolist()

Oficinas = df['OFICINA_AFO'].unique().tolist()

df_fam_canales=df[['FAMILIA_AFO','CANAL']].drop_duplicates()

df_dicc_Material=df[["MATERIAL_AFO","FAMILIA_AFO","CATEGORIA_AFO","NEGOCIO_AFO"
                            ]].drop_duplicates()

df_dicc_familia_Cat= df[["FAMILIA_AFO","CATEGORIA_AFO"]
                            ].drop_duplicates()
df_dicc_familia_Cat=pd.DataFrame(df_dicc_familia_Cat)
df_dicc_familia_Cat=df_dicc_familia_Cat.reset_index().drop(columns=["index"])
df_dicc_familia_Cat


,FAMILIA_AFO,CATEGORIA_AFO
0,1001001001 Don Vittorio Colecci,1001001 Pastas
1,1001001002 Don Vittorio Clasico,1001001 Pastas
2,1001001004 Alianza Clasico,1001001 Pastas
3,1001001005 Nicolini Clasico,1001001 Pastas
4,1001001011 Lavaggi Clasico,1001001 Pastas
...,...,...
263,5016105B37 Amaras Bendito Clima,5016105 Acondicionador
264,5016105B76 Amaras Magia Andina,5016105 Acondicionador
265,5016114B38 Amaras Reparacion Di,5016114 Tratamientos
266,5016114B39 Amaras Bendito Clima,5016114 Tratamientos


In [4]:
# DATAFRAME SI average ultimos 3

df=df.drop(columns=["Categoría","Cat2","Oficina de ventas","OF2","Negocio","NEG2","Familia","FAM2","Cliente actual","CA2",
                    "Territorio","TER2","Material histórico","MAT2"])



N = 3

df_num = df.select_dtypes(include=[np.float64])

count= df_num.shape[1]

avg = df.iloc[:,-N:]
print(avg)
avg = avg.mean(axis=1)
print(avg)
df_avg=df.iloc[:,:-count]
df_avg[f"average {N}"]= avg
print(df_avg[f"average {N}"].sum())

       07.2023  08.2023  09.2023
0          NaN      NaN     0.01
1       5.7000  2.70000     3.60
2       0.2050  0.08500      NaN
3       0.0300  0.08500     0.10
4       0.6480  0.77760      NaN
...        ...      ...      ...
82795   0.0891      NaN      NaN
82796      NaN      NaN      NaN
82797      NaN      NaN      NaN
82798      NaN      NaN      NaN
82799      NaN  0.01068      NaN

[82800 rows x 3 columns]
0        0.010000
1        4.000000
2        0.145000
3        0.071667
4        0.712800
           ...   
82795    0.089100
82796         NaN
82797         NaN
82798         NaN
82799    0.010680
Length: 82800, dtype: float64
62017.900791666674


In [5]:
df_subset={}
df_subset_perc={}


for ofi in Oficinas:
    for fam in familias:
        df_subset[str(ofi)+"_"+str(fam)]= df_avg.loc[(df_avg["OFICINA_AFO"]==ofi) & (df_avg["FAMILIA_AFO"]==fam)]
        total =df_subset[str(ofi)+"_"+str(fam)].groupby(by=["CANAL","OFICINA_AFO","CLIENTE_AFO","TERRITORIO_AFO","MATERIAL_AFO"
                                ])[f"average {N}"].sum().sum()
        df_subset_perc[str(ofi)+"_"+str(fam)]= df_subset[str(ofi)+"_"+str(fam)].groupby(by=["CANAL","OFICINA_AFO","CLIENTE_AFO","TERRITORIO_AFO","MATERIAL_AFO"
                                ])[f"average {N}"].sum()/total


In [6]:
print(df_subset_perc["PE01 CM-LIMA_2008044274 Sapolio Auto Cera"])

Series([], Name: average 3, dtype: float64)


In [7]:
#IBP
df_demanda.rename(columns={'Familia Desc.':'FAMILIA_AFO'},inplace=True)
df_demanda.rename(columns={'Categoría Desc.':'CATEGORIA_AFO'},inplace=True)
df_demanda.rename(columns={'Oficina de Ventas Desc. S4':'OFICINA_AFO'},inplace=True)

columnas_demanda=df_demanda.columns
columna_predicción_demanda= columnas_demanda[-1]

df_demanda["FAMILIA_AFO"]=df_demanda["FAMILIA_AFO"].replace({"-":" "},regex=True)
df_demanda["CATEGORIA_AFO"]=df_demanda["CATEGORIA_AFO"].replace({"-":" "},regex=True)

df_demanda_oficina_familia =df_demanda.groupby(by=["OFICINA_AFO","FAMILIA_AFO"],as_index=False
                            )[columna_predicción_demanda].sum()
df_demanda_oficina_familia

,OFICINA_AFO,FAMILIA_AFO,Split total
0,PE01 CM-LIMA,1001001002 Don Vittorio Clasico,926.423505
1,PE01 CM-LIMA,1001001004 Alianza Clasico,4.549625
2,PE01 CM-LIMA,1001001004v Alianza Clasico,18.333086
3,PE01 CM-LIMA,1001001005 Nicolini Clasico,419.875740
4,PE01 CM-LIMA,1001001008 Espiga de Oro Clasic,452.607171
...,...,...,...
869,PE24 CM-N/S CH-ORIENTE-CE,5016105B37 Amaras Bendito Clima,0.490967
870,PE24 CM-N/S CH-ORIENTE-CE,5016105B76 Amaras Magia Andina,0.976243
871,PE24 CM-N/S CH-ORIENTE-CE,5016114B38 Amaras Reparacion Di,0.156349
872,PE24 CM-N/S CH-ORIENTE-CE,5016114B39 Amaras Bendito Clima,0.112604


In [8]:
#SPLIT FINAL

familias_ibp= pd.unique(df_demanda["FAMILIA_AFO"]).tolist()
oficinas_ibp= pd.unique(df_demanda["OFICINA_AFO"]).tolist()

df_final=pd.DataFrame()
for ofi in oficinas_ibp:
    for fam in familias_ibp:
        key = str(ofi) + "_" + str(fam)
        print(key)

        if key in df_subset_perc and not df_subset_perc.get(key, pd.DataFrame()).empty:
            df_subset_escogido=df_subset_perc[str(ofi)+"_"+str(fam)].reset_index()
            df_x= df_demanda[(df_demanda["OFICINA_AFO"]==str(ofi))&(df_demanda["FAMILIA_AFO"]==str(fam))]
            total_oficina_familia=df_x["Split total"].values
            if len(total_oficina_familia) > 0:
                df_subset_escogido[f"average {N}"]=df_subset_escogido[f"average {N}"]* total_oficina_familia
                df_final=pd.concat([df_final,df_subset_escogido],ignore_index=True)
            else:
                print("total_oficina_familia está vacío, saltando...")
            print("-----------------------------------------------------------------")
        else:
            print("df_subset_perc vacío o no existe para esta combinación, saltando...")




df_final = df_final.merge(df_dicc_Material,right_on="MATERIAL_AFO",left_on="MATERIAL_AFO")
print(df_final)

PE01 CM-LIMA_1001001002 Don Vittorio Clasico
-----------------------------------------------------------------
PE01 CM-LIMA_1001001004 Alianza Clasico
-----------------------------------------------------------------
PE01 CM-LIMA_1001001005 Nicolini Clasico
-----------------------------------------------------------------
PE01 CM-LIMA_1001001011 Lavaggi Clasico
-----------------------------------------------------------------
PE01 CM-LIMA_1001001004v Alianza Clasico
-----------------------------------------------------------------
PE01 CM-LIMA_1001001008 Espiga de Oro Clasic
-----------------------------------------------------------------
PE01 CM-LIMA_1001001009 Victoria Clasico
df_subset_perc vacío o no existe para esta combinación, saltando...
PE01 CM-LIMA_1001002013 Favorita Domestica
-----------------------------------------------------------------
PE01 CM-LIMA_1001002014 Blanca Flor Domestic
-----------------------------------------------------------------
PE01 CM-LIMA_1001002018

In [9]:

df_final.to_excel("C:/Renato/script_atenea/Split2/df_split 3.xlsx"
,sheet_name="Hoja1")